# Twitter Streaming And Email Alert

In [ ]:
# Install library
!pip install smtplib tweepy

### 1. Setup the email

In [1]:
import smtplib

def send_email(user, pwd, recipient, subject, body):    

    FROM = user
    TO = recipient if isinstance(recipient, list) else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()  # upgrade the connection to secure
        server.login(user, pwd)
        server.sendmail(FROM, TO, message)
        server.close()
        print ('successfully sent the mail')
    except:
        print ("failed to send mail")

gmail_user = 'fwayne2012@gmail.com'
gmail_password = 'Cnrc5068'
to = ['derrick.wang@dialyspa.com']
subject = 'Sent by Python'

### 2. Setup Twitter Connection

In [2]:
import re
import pandas as pd
import tweepy
import pandas
import json # The API returns JSON formatted text
import os
os.chdir(r'C:\Users\dwang\Documents\Tech Docs\Natural Language Processing\Twitter')

# Store OAuth authentication credentials - get at https://developer.twitter.com/en/apps
access_token = "1090469255995363328-Zp0IczEkb7OZV9zyFx29WCf36BaHDu"
access_token_secret = "Va2DUqrIez2AR6ksOrjmeHiynR8OJhLd4d56fldkn4uhM"
consumer_key = "Vb2SjXqcB1mr0OBeqapXKHfjT"
consumer_secret = "S7CCwSE35ImaDJ7GOVeAgaExKQT9kVeJJ1gI0SMZE6LM97DOJn"


# Pass OAuth details to tweepy's OAuth handler
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

### 3. Define the twitter streaming

In [15]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')


class MyStreamListener(tweepy.StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        #self.file = open(OUTPUT_FILE, "w")

    def on_status(self, status):
        tweet = status._json
        UserId = tweet['user']['id']
        
        if str(UserId) in [TwitterID_DT, TwitterID_DW, TwitterID_WSJ]:                     
            tweet_id = tweet['id']
            TweetTime = tweet['created_at']       
            try:
                Text = tweet['extended_tweet']['full_text']
                Text = re.sub(r'@+[a-zA-Z]+','',Text)
                Text = re.sub(r'#+[a-zA-Z]+','',Text)
                Text = re.sub(r'\n','', Text)
            except:
                Text = tweet['text']
                Text = re.sub(r'@+[a-zA-Z]+','',Text)
                Text = re.sub(r'#+[a-zA-Z0-9]+','',Text)
                Text = re.sub(r'\n','', Text)
                
            #UserName = tweet['user']['name']        
            UserScreenName = tweet['user']['screen_name']       
            subject = UserScreenName + ' newly tweeted.'
            
            body = deEmojify(Text)  
            RstList.append([TweetTime, tweet_id, deEmojify(Text), UserScreenName, UserId])
            send_email(gmail_user, gmail_password, to, subject, body)
        
            self.num_tweets += 1
            print('Numer of tweets captured so far: {}'.format(self.num_tweets))
            
            if self.num_tweets < TWEETS_TO_CAPTURE:                
                return True  # continue streaming
            else:                
                return False  # stop streaming

    def on_error(self, status):
        print(status)

### 4. Run the streaming and send email if DT or DW tweets

In [ ]:
TWEETS_TO_CAPTURE = 10
RstList = []

TwitterID_DW = '1090469255995363328'
TwitterID_DT = '25073877'
TwitterID_WSJ = '28164923' 

# Initialize Stream listener
l = MyStreamListener()

# Create you Stream object with authentication
stream = tweepy.Stream(auth, l)

# Filter Twitter Streams to capture data by the keywords or userID:
stream.filter(follow= [TwitterID_DW, TwitterID_DT, TwitterID_WSJ])

successfully sent the mail
Numer of tweets captured so far: 1
successfully sent the mail
Numer of tweets captured so far: 2
successfully sent the mail
Numer of tweets captured so far: 3


In [17]:
Rst = pd.DataFrame(RstList, columns = ['DateTime', 'TweetId','Text', 'UserName', 'UserID'])
Rst['DateTime'] = pd.to_datetime(Rst['DateTime'])
Rst['DateTime'] = Rst['DateTime'].dt.tz_convert('America/Chicago')
#Rst['DateTime'] = Rst['DateTime'].
Rst

,DateTime,TweetId,Text,UserName,UserID
0,2019-10-10 14:16:25-05:00,1182374344862904333,sdfgshj rsrtsgf,derrickfwang,1090469255995363328
1,2019-10-10 14:17:05-05:00,1182374511267721216,Cost-of-living increase smaller than the 2.8% ...,WSJmarkets,28164923
2,2019-10-10 14:40:06-05:00,1182380304461451265,With trading as a lure to gather more customer...,WSJmarkets,28164923
3,2019-10-10 14:49:46-05:00,1182382736205668353,RT : Heard on the Street: Deltas forecasts for...,WSJmarkets,28164923
4,2019-10-10 15:06:08-05:00,1182386854257184768,A management change at both car makers could p...,WSJmarkets,28164923
5,2019-10-10 15:07:12-05:00,1182387122382262273,RT : : President Trumps Thursday tweet about m...,WSJmarkets,28164923
6,2019-10-10 15:07:30-05:00,1182387197066039298,We defeated 100% of the ISIS Caliphate and no ...,realDonaldTrump,25073877
7,2019-10-10 15:07:30-05:00,1182387198164946944,....We have one of three choices: Send in thou...,realDonaldTrump,25073877
8,2019-10-10 15:23:05-05:00,1182391121021943809,Federal Reserve gives large banks a break on p...,WSJmarkets,28164923
9,2019-10-10 15:42:01-05:00,1182395885273206785,Shareholders should expect more pain as the ut...,WSJmarkets,28164923


### 5. Sending Aleart email to notify that streaming stopped or completed

In [7]:
body = 'Streaming completed or stopped.'
send_email(gmail_user, gmail_password, to, subject, body)

successfully sent the mail
